# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846877480532                   -0.70    4.5         
  2   -7.852325331859       -2.26       -1.53    1.0   16.8ms
  3   -7.852615087541       -3.54       -2.56    1.5   18.5ms
  4   -7.852645996189       -4.51       -2.88    2.2   22.2ms
  5   -7.852646494101       -6.30       -3.16    1.2   17.7ms
  6   -7.852646679542       -6.73       -4.03    1.0   17.1ms
  7   -7.852646686379       -8.17       -5.24    1.8   20.3ms
  8   -7.852646686725       -9.46       -5.52    2.0   21.5ms
  9   -7.852646686729      -11.35       -6.03    1.0   17.4ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846892306702                   -0.70           4.8         
  2   -7.852526077119       -2.25       -1.64   0.80    2.0    215ms
  3   -7.852635604632       -3.96       -2.73   0.80    1.0   16.4ms
  4   -7.852646589993       -4.96       -3.24   0.80    2.2   21.5ms
  5   -7.852646681909       -7.04       -4.09   0.80    1.0   16.1ms
  6   -7.852646686272       -8.36       -4.89   0.80    1.2   17.4ms
  7   -7.852646686726       -9.34       -5.61   0.80    2.2   22.3ms
  8   -7.852646686729      -11.42       -6.63   0.80    1.2   17.6ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.347349e+01     3.682284e+00
 * time: 0.046631813049316406
     1     1.321976e+00     1.912984e+00
 * time: 0.21985101699829102
     2    -1.380205e+00     2.117997e+00
 * time: 0.23807883262634277
     3    -3.417269e+00     1.914781e+00
 * time: 0.2639789581298828
     4    -4.561229e+00     1.841901e+00
 * time: 0.28960204124450684
     5    -6.273164e+00     1.364865e+00
 * time: 0.31516385078430176
     6    -7.098037e+00     6.954454e-01
 * time: 0.34082603454589844
     7    -7.434242e+00     6.056245e-01
 * time: 0.35866785049438477
     8    -7.662328e+00     2.303405e-01
 * time: 0.37654995918273926
     9    -7.755361e+00     1.265019e-01
 * time: 0.39444899559020996
    10    -7.808950e+00     7.795294e-02
 * time: 0.4123709201812744
    11    -7.828093e+00     5.290081e-02
 * time: 0.4303770065307617
    12    -7.840397e+00     3.037965e-02
 * time: 0.4482388496398926
    13    -7.844348e+00     1.929027e-02
 * time: 0.

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846853555469                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645920843                   -1.64         
  2   -7.852646686730       -6.12       -3.71    1.51s
  3   -7.852646686730      -13.30       -7.25    151ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 6.07445843437428e-7
|ρ_newton - ρ_scfv| = 4.4641731259312676e-7
|ρ_newton - ρ_dm|   = 0.0197758376321207
